Imports

In [1]:
import json
import requests
from bs4 import BeautifulSoup

2.Sending Http request to access first page with opinions


In [2]:
url="https://www.ceneo.pl/84514582#tab=reviews"
response = requests.get(url)
response.status_code

200

3.Checking the code of http response

In [3]:
response.status_code

200

4.Parse the html code of first age with opinions

In [4]:
page_dom = BeautifulSoup(response.text, 'html.parser')
print(response.text)



<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7  non-direct ab-inactive" lang="pl"> <![endif]-->
<!--[if IE 7]> <html class="no-js lt-ie9 lt-ie8  non-direct ab-inactive" lang="pl"> <![endif]-->
<!--[if IE 8]> <html class="no-js lt-ie9  non-direct ab-inactive" lang="pl"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js  non-direct ab-inactive" lang="pl">
<!--<![endif]-->
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<link rel="preconnect" href="https://image.ceneo.pl">
<link rel="preconnect" href="https://image.ceneostatic.pl">
<link rel="preconnect" href="https://www.google-analytics.com">
<link rel="preconnect" href="https://www.googleadservices.com">
<link rel="preconnect" href="https://adservice.google.com">
<link rel="preconnect" href="https://www.google.pl">
<link rel="preconnect" href="https://www.google.com">
<link rel="preconnect" href="https://cdn.ampproject.org">


 <title>Urządzenie wielofunkcyjne HP Sma

5.Extract required data from parsed code

In [8]:
opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
opinion = page_dom.select_one("div.js_product-review:not(.user-post--highlight)")
print(type(page_dom))
print(type(opinions))
print(len(opinions))
print(type(opinion))
print(opinion.prettify())

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>
10
<class 'bs4.element.Tag'>
<div class="user-post user-post__card js_product-review" data-entry-id="12535429">
 <div class="user-post__header">
  <div class="js_lazy user-post__avatar user-rank__avatar" data-bg="/Content/img/account/avatar/0.svg">
  </div>
 </div>
 <div class="user-post__body">
  <div class="user-post__content">
   <div>
    <span class="user-post__author-name">
     g...6
    </span>
    <span class="user-post__score">
     <span class="screen-reader-text">
      Ocena:
     </span>
     <span class="score-container score-container--s js_score-container">
      <span class="score-marker score-marker--s" style="width: 60.00%;">
      </span>
     </span>
     <span class="user-post__score-count">
      3/5
     </span>
    </span>
   </div>
   <div class="mb-3">
    <span class="verified-purchase">
     Zaufana Opinia potwierdzona zakupem
    </span>
    <span class="user-post__published">
     <span class="u

In [25]:
opinion_id = opinion["data-entry-id"]
author = opinion.select_one("span.user-post__author-name").get_text()
try:
    recomendation = opinion.select_one("span.user-post__author-recomendation > em").get_text()
except AttributeError:
    recomendation = None
stars = opinion.select_one("span.user-post__score-count").get_text()
content = opinion.select_one("div.user-post__text").get_text()
pros = [p.get_text() for p in opinion.select("div.review-feature__item--positive")]
cons = [c.get_text() for c in opinion.select("div.review-feature__item--negative")]
vote_yes = opinion.select_one("button.vote-yes[data-total-vote]")
vote_no = opinion.select_one("button.vote-no[data-total-vote]")
published = opinion.select_one("span.user-post__published > time:nth-child(1)[datatime]")
purchased = opinion.select_one("span.user-post__published > time:nth-child(2)[datetime]")
print(opinion_id, author, recomendation, stars, content, pros, cons, vote_yes, vote_no, published, purchased, sep='\n')

13243464

m...i

Nie polecam
0,5/5
Niestety pierwsze wrazenie po uruchomieniu sprzetu negatywne. Odrazu trzeba bylo skladac reklamacje poniewaz jakosc druku odbiegała od normy. Po zgloszeniu okazalo sie ze jest uszkodzona glowica. Zobaczymy jak przyjdzie nowa glowica czy cos to da. Jaknarazie to wielka porażka a myslalam ze sprzet za ta cene bedzie działał bez zarzutu. Niestety pomyliłam się. 
[]
[]
<button class="vote-yes js_product-review-vote js_vote-yes" data-new-icon="vote-up" data-product-id="84514582" data-review-id="13243464" data-total-vote="8" data-url="SetOpinionVote" data-vote="1" data-voted="false"><span id="votes-yes-13243464">8</span></button>
<button class="vote-no js_product-review-vote js_vote-no" data-new-icon="vote-down" data-product-id="84514582" data-review-id="13243464" data-total-vote="4" data-url="SetOpinionVote" data-vote="0" data-voted="false"><span id="votes-no-13243464">4</span></button>
None
<time datetime="2020-10-16 15:51:18">po 4 dniach</time>


In [35]:
opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
for opinion in opinions:
    opinion_id = opinion["data-entry-id"]
    author = opinion.select_one("span.user-post__author-name").get_text()
    try:
        recomendation = opinion.select_one("span.user-post__author-recomendation > em").get_text()
    except AttributeError:
        recomendation = None
    stars = opinion.select_one("span.user-post__score-count").get_text()
    content = opinion.select_one("div.user-post__text").get_text()
    pros = [p.get_text() for p in opinion.select("div.review-feature__item--positive")]
    cons = [c.get_text() for c in opinion.select("div.review-feature__item--negative")]
    vote_yes = opinion.select_one("button.vote-yes[data-total-vote]")
    vote_no = opinion.select_one("button.vote-no[data-total-vote]")
    published = opinion.select_one("span.user-post__published > time:nth-child(1)[datatime]")
    purchased = opinion.select_one("span.user-post__published > time:nth-child(2)[datetime]")
    try:
       purchased = opinion.select_one("span.user-post__published > time:nth-child(2)[datetime]")
    except TypeError:
        purchased = None
    print(opinion_id, author, recomendation, stars, content, pros, cons, vote_yes, vote_no, published, purchased, sep='\n')

12535429

g...6

None
3/5
Instalacja łatwa. Oprogramowanie HP Smart bezproblemowe. Zwrot 100zl Cashback z Promocji po kilku dniach od zakupu. Skanuje. Drukuje. 
Przez pierwsze 2-3 dni wszystko w porządku, wydruki czarne i kolorowe bardzo dobre. Po 3 dniu, zupełnie nagle, drukarka zaczyna drukować białe poziome kreski na wydrukach kolorowych. W czarnym bardzo dobrze. Wydruki w najlepszej jakości są bardzo dobre. Wydruki w jakości normalnej i roboczej są moim zdaniem wadliwe. Kupiłem 2 drukarki. Jedną udało mi się zwrócić w ramach 14 zwrotu. Druga niestety została. 
Producent HP raz uznał gwarancję. HP przysłałi nową głowicę kolorową, a jak problem nie został rozwiązany, całą nową drukarkę. Trzecia drukarka tak samo, drukuje białe kreski na wydrukach kolorowych. 
Zgłoszone ponownie do HP, tym razem w ocenie wsparcia HP, drukarki atramentowe tak drukują... Pozostawię to bez komentarza. Chciałem zgłosić w ramach rękojmi, niezgodność produktu z umową (reklamują outstanding printing quality)

6.If there are more pages, move to the next page and repeat steps 2-6 for it

In [32]:
try:
   next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination_next")['href']
except TypeError:
   next_page = None

In [34]:
next_page = https://www.ceneo.pl/84514582#tab=reviews
all_opinions = {}
while next_page:
    response = requests.get(next_page)
    if response.status_code == 200:
        print('OK')
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(opinions))
    for opinion in opinions:
        single_opinion = {}
        single_opinion["opinion_id"] = opinion["data-entry-id"]
        single_opinion["author"] = opinion.select_one("span.user-post__author-name").get_text()
    try:
        single_opinion["recomendation"] = opinion.select_one("span.user-post__author-recomendation > em").get_text()
    except AttributeError:
        single_opinion["recomendation"] = None
    single_opinion["stars"] = opinion.select_one("span.user-post__score-count").get_text()
    single_opinion["content"] = opinion.select_one("div.user-post__text").get_text()
    single_opinion["pros"] = [p.get_text() for p in opinion.select("div.review-feature__item--positive")]
    single_opinion["cons"] = [c.get_text() for c in opinion.select("div.review-feature__item--negative")]
    single_opinion["vote_yes"] = opinion.select_one("button.vote-yes[data-total-vote]")
    single_opinion["vote_no"] = opinion.select_one("button.vote-no[data-total-vote]")
    single_opinion["published"] = opinion.select_one("span.user-post__published > time:nth-child(1)[datatime]")
    purchased = opinion.select_one("span.user-post__published > time:nth-child(2)[datetime]")
    try:
        single_opinion["purchased"] = opinion.select_one("span.user-post__published > time:nth-child(2)[datetime]")
    except TypeError:
        single_opinion["purchased"] = None
    print(opinion_id, author, recomendation, stars, content, pros, cons, vote_yes, vote_no, published, purchased, sep='\n')
    all_opinions.append{single_opinion}
try:
    next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination_next")["href"]
except TypeError:
            next_page = None

7. Save extracted data